In [2]:
import numpy as np
from skimage import exposure
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy import ndimage as ndi
from cellpose import models
import napari
import os
import cv2
import matplotlib.pyplot as plt

# Load your image
os.chdir("E:/data_for_seg/z_projection/receed0")
files = os.listdir()

# Load the first image in the directory in grayscale
image = cv2.imread(files[1], cv2.IMREAD_GRAYSCALE)

spacing = np.array([0.26, 0.26])

viewer = napari.view_image(
    image,
    scale=spacing,
    ndisplay=2,
)

# Get the selected image layer
selected_layer = viewer.layers.selection.active

# Convert the image to float
image = selected_layer.data.astype(float)

# Enhance the intensity of the cells using histogram equalization
equalized_image = exposure.equalize_hist(image)

# Add the enhanced image to the viewer
viewer.add_image(equalized_image, name='Enhanced Image')

# Segment the cells in the enhanced image using the Cellpose method
model = models.Cellpose(gpu=False, model_type='cyto')

# Run the model on the equalized image
masks, flows, styles, diams = model.eval(equalized_image, diameter=None, channels=[0,0])

# Add the segmented image to the viewer
viewer.add_labels(masks, name='Segmented Cells')

<Labels layer 'Segmented Cells' at 0x2c5a177b910>